# Homework

The goal of this homework is to get familiar with MLflow, the tool for experiment tracking and model management.

## Q1. Install MLflow

To get started with MLflow you'll need to install the MLflow Python package.

For this we recommend creating a separate Python environment, for example, you can use conda environments, and then install the package there with `pip` or `conda`.

Once you installed the package, run the command `mlflow --version` and check the output.

In [1]:
!conda install --quiet --yes mlflow

Retrieving notices: ...working... done
Channels:
 - conda-forge
Platform: linux-64
Solving environment: ...working... done

## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - mlflow


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    aiohappyeyeballs-2.4.0     |     pyhd8ed1ab_0          17 KB  conda-forge
    aiohttp-3.10.5             |  py311h9ecbd09_1         814 KB  conda-forge
    aiosignal-1.3.1            |     pyhd8ed1ab_0          12 KB  conda-forge
    aniso8601-9.0.1            |     pyhd8ed1ab_0          38 KB  conda-forge
    bcrypt-4.2.0               |  py311h9e33e62_1         243 KB  conda-forge
    cachetools-5.5.0           |     pyhd8ed1ab_0          14 KB  conda-forge
    click-8.1.7                |unix_pyh707e725_0          82 KB  conda-forge
    cloudpickle-3.0.0          |     pyhd8ed1ab_0          24 KB  conda-forge
    databric

What's the version that you have?

In [1]:
!mlflow --version

mlflow, version 2.16.2


## Q2. Download and preprocess the data

We'll use the Green Taxi Trip Records dataset to predict the duration of each trip.

Download the data for January, February and March 2023 in parquet format from [here](https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page).

In [1]:
!mkdir -p data
!curl https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-01.parquet -o data/green_tripdata_2023-01.parquet
!curl https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-02.parquet -o data/green_tripdata_2023-02.parquet
!curl https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-03.parquet -o data/green_tripdata_2023-03.parquet

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1393k  100 1393k    0     0  3315k      0 --:--:-- --:--:-- --:--:-- 3310k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1497k  100 1497k    0     0  3791k      0 --:--:-- --:--:-- --:--:-- 3801k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1690k  100 1690k    0     0  3640k      0 --:--:-- --:--:-- --:--:-- 3635k


Use the script `preprocess_data.py` located in the folder homework to preprocess the data.

The script will:

- load the data from the folder <TAXI_DATA_FOLDER> (the folder where you have downloaded the data),
- fit a DictVectorizer on the training set (January 2023 data),
- save the preprocessed datasets and the DictVectorizer to disk.

Your task is to download the datasets and then execute this command:

```shell
python preprocess_data.py --raw_data_path <TAXI_DATA_FOLDER> --dest_path ./
```

*Tip: go to homework folder before executing the command and change the value of <TAXI_DATA_FOLDER> to the location where you saved the data.*

In [2]:
!python preprocess_data.py --raw_data_path ./data --dest_path ./output

How many files were saved to `OUTPUT_FOLDER`?

In [3]:
!ls ./output/ | wc -l

4


## Q3. Train a model with autolog

We will train a `RandomForestRegressor` (from Scikit-Learn) on the taxi dataset.

We have prepared the training script `train.py` for this exercise, which can be also found in the folder homework.

The script will:

- load the datasets produced by the previous step,
- train the model on the training set,
- calculate the RMSE score on the validation set.

Your task is to modify the script to enable **autologging** with MLflow, execute the script and then launch the MLflow UI to check that the experiment run was properly tracked.

*Tip 1: don't forget to wrap the training code with a with `mlflow.start_run()`: statement as we showed in the videos.*

*Tip 2: don't modify the hyperparameters of the model to make sure that the training will finish quickly.*

`mlflow ui --backend-store-uri=sqlite:///mlflow.db -h 0.0.0.0 -p 8080`

In [4]:
!python train.py

2024/09/30 13:46:28 INFO mlflow.tracking.fluent: Experiment with name 'mlops-course-week2' does not exist. Creating a new experiment.
Run ID: 2e4ab843712b475586a10cf92c68c4b1
/home/igor_lysak/miniconda3/envs/mlops/lib/python3.11/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


What is the value of the `min_samples_split` parameter:

In [5]:
import mlflow
from mlflow.tracking import MlflowClient

mlflow.set_tracking_uri("sqlite:///mlflow.db")

client = MlflowClient()
experiment = client.get_experiment_by_name("mlops-course-week2")
best_run = client.search_runs(
    experiment_ids=experiment.experiment_id,
    order_by=["metrics.test_rmse ASC"],
)[0]
print(f"{best_run.data.params['min_samples_split']}")

2


## Q4. Launch the tracking server locally

Now we want to manage the entire lifecycle of our ML model. In this step, you'll need to launch a tracking server. This way we will also have access to the model registry.

Your task is to:

- launch the tracking server on your local machine,
- select a SQLite db for the backend store and a folder called artifacts for the artifacts store.
- You should keep the tracking server running to work on the next two exercises that use the server.

`mlflow server --backend-store-uri=sqlite:///mlflow.db --default-artifact-root ./artifacts -h 0.0.0.0 -p 8080`

In addition to `backend-store-uri`, what else do you need to pass to properly configure the server?

`default-artifact-root`

## Q5. Tune model hyperparameters

Now let's try to reduce the validation error by tuning the hyperparameters of the RandomForestRegressor using hyperopt. We have prepared the script hpo.py for this exercise.

Your task is to modify the script `hpo.py` and make sure that the validation RMSE is logged to the tracking server for each run of the hyperparameter optimization (you will need to add a few lines of code to the `objective` function) and run the script without passing any parameters.

After that, open UI and explore the runs from the experiment called random-forest-hyperopt to answer the question below.

*Note: Don't use autologging for this exercise.*

The idea is to just log the information that you need to answer the question below, including:

- the list of hyperparameters that are passed to the objective function during the optimization,
- the RMSE obtained on the validation set (February 2023 data).

In [6]:
!python hpo.py

2024/09/30 13:47:10 INFO mlflow.tracking.fluent: Experiment with name 'random-forest-hyperopt' does not exist. Creating a new experiment.
  0%|                                    | 0/15 [00:00<?, ?trial/s, best loss=?]/home/igor_lysak/miniconda3/envs/mlops/lib/python3.11/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(

2024/09/30 13:47:20 INFO mlflow.tracking._tracking_service.client: 🏃 View run puzzled-stoat-524 at: http://localhost:8080/#/experiments/2/runs/6d4c68293a984d6c88099a7acb5d4903.

2024/09/30 13:47:20 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:8080/#/experiments/2.

  7%|▋          | 1/15 [00:09<02:12,  9.46s/trial, best loss: 5.370086069268862]/home/igor_lysak/miniconda3/envs/mlops/lib/python3.11/site-packages/sklearn/metrics/_regression.py:492: Fu

What's the best validation RMSE that you got?

In [7]:
experiment = client.get_experiment_by_name("random-forest-hyperopt")
best_run = client.search_runs(
    experiment_ids=experiment.experiment_id,
    order_by=["metrics.test_rmse ASC"],
)[0]
print(f"{best_run.data.metrics['rmse']:0.3f}")

5.355


## Q6. Promote the best model to the model registry

The results from the hyperparameter optimization are quite good. So, we can assume that we are ready to test some of these models in production. In this exercise, you'll promote the best model to the model registry. We have prepared a script called `register_model.py`, which will check the results from the previous step and select the top 5 runs. After that, it will calculate the RMSE of those models on the test set (March 2023 data) and save the results to a new experiment called `random-forest-best-models`.

Your task is to update the script `register_model.py` so that it selects the model with the lowest RMSE on the test set and registers it to the model registry.

*Tip 1: you can use the method `search_runs` from the `MlflowClient` to get the model with the lowest RMSE,*

*Tip 2: to register the model you can use the method `mlflow.register_model` and you will need to pass the right model_uri in the form of a string that looks like this: `"runs:/<RUN_ID>/model"`, and the name of the model (make sure to choose a good one!).*

In [8]:
!python register_model.py

2024/09/30 13:49:07 INFO mlflow.tracking.fluent: Experiment with name 'random-forest-best-models' does not exist. Creating a new experiment.
2024/09/30 13:49:08 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'
/home/igor_lysak/miniconda3/envs/mlops/lib/python3.11/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/home/igor_lysak/miniconda3/envs/mlops/lib/python3.11/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
2024/09/30 13:49:17 INFO mlflow.tracking._tracking_service.client: 🏃 View run vaunted-squid-723 at: http://localhost:8

What is the test RMSE of the best model?

In [9]:
experiment = client.get_experiment_by_name("random-forest-best-models")
best_run = client.search_runs(
    experiment_ids=experiment.experiment_id,
    order_by=["metrics.test_rmse ASC"],
)[0]
print(f"{best_run.data.metrics['test_rmse']:0.3f}")

5.567


## Summary

### Q1. Install MLflow

`mlflow, version 2.16.2`

### Q2. Download and preprocess the data

4

### Q3. Train a model with autolog

2

### Q4. Launch the tracking server locally

`default-artifact-root`

### Q5. Tune model hyperparameters

5.335

### Q6. Promote the best model to the model registry

5.567
